In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
##Load Dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Row number,Surane,CustomerId will wont do any impact

In [4]:
#Preprocess the data
#Drop irrelavent features which we can directly identified
data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Geography and Gender are the Categorical Features.
# Applying Label Encoading for Categorical Features
Label_Encoder_Gender = LabelEncoder()
data["Gender"] = Label_Encoder_Gender.fit_transform(data["Gender"])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data["Geography"].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
#One hot encode Geography Column
from sklearn.preprocessing import OneHotEncoder

onehotencoder_geo=OneHotEncoder()
geo_encoded_value = onehotencoder_geo.fit_transform(data[["Geography"]])
geo_encoded_value

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
#cONVERTING TO NUMPY ARRAY
onehotencoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
#Converting to Data Frame and Assignig columns
geo_encoded_df=pd.DataFrame(geo_encoded_value.toarray(),columns=onehotencoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
#Combining all the columns with original Data
data = pd.concat([data,geo_encoded_df],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
#Geography is no way neede
data=data.drop("Geography",axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


#### Pickling the One hot Encoding and Label Encoding

In [12]:
# Save the encoders ans sscaler ,, dumping label encoader to a file 

with open('Label_Encoder_Gender.pkl','wb') as file:
    pickle.dump(Label_Encoder_Gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file2:
    pickle.dump(onehotencoder_geo,file2)

In [13]:
#Dividing the Features into Independent and Dependent Features

X=data.drop("Exited",axis=1)
y=data["Exited"]

#Split the data in training and Testing side

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [14]:
### Pickling Scaler file
with open('scaler.pkl','wb') as file3:
    pickle.dump(scaler,file3)

In [15]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


### ANN IMPLEMENTATION

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [17]:
print(X_train.shape)
print(X_train.shape[1])

(8000, 12)
12


In [18]:
#building ANN Model

model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),#HL1 connected to input layer
    Dense(32,activation="relu"), #HL2 
    Dense(1,activation='sigmoid') #output layer
])


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
#Keras optimizers
#method-1
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
binary_loss = tf.keras.losses.BinaryCrossentropy()
binary_loss

In [21]:
## Compile the model with optimizers
#method-2
#---> model.compile(optimizer="adam",loss="binary_crossentrophy",metric=["accuracy"])
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])

In [22]:
## Setup the tensorBoard
#Capturing and Visualizing the Logs ---> Tensor Board

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%%S")  #Logfile text format
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [23]:
#Set up Early Stopping
# Where a scenario that my model has trained best for 20 EPOCHS, so i need to early stop the model even though we have some remaining Epochs
#Continious monitoring of loss in mandatory.--> for min gradient

early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)


In [24]:
### Training the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    
)

Epoch 1/100


250/250 [==============================] - 4s 5ms/step - loss: 0.3986 - accuracy: 0.8330 - val_loss: 0.3621 - val_accuracy: 0.8410
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3593 - accuracy: 0.8499 - val_loss: 0.3609 - val_accuracy: 0.8560
Epoch 3/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3495 - accuracy: 0.8591 - val_loss: 0.3438 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3429 - accuracy: 0.8586 - val_loss: 0.3461 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3413 - accuracy: 0.8605 - val_loss: 0.3445 - val_accuracy: 0.8595
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3395 - accuracy: 0.8591 - val_loss: 0.3492 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3377 - accuracy: 0.8618 - val_loss: 0.3674 - val_accuracy: 0.85

#### Saving the model with h5 file

In [25]:
#h5 file is compatable with Keras

model.save('model.h5')

m:\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
##Load Tensor Board Extension with line magic
%load_ext tensorboard

In [27]:
'''
#%tensorboard --logdir logs/fit
%tensorboard --logdir logs/fit20250130-1406%S
'''

'\n#%tensorboard --logdir logs/fit\n%tensorboard --logdir logs/fit20250130-1406%S\n'

#### Load the Pickle Files

In [28]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [29]:
#Load the all Models
#ANN model,scale pickle,onehot pickle


In [30]:
model=load_model('model.h5')

##load the encoder and Scler
with open('Label_Encoder_Gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

##### Predictions

In [31]:
#Example Input Data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':5000
}

##### Geography Pickle

In [32]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

m:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [33]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,5000


##### Categorical(Gender)

In [34]:
input_df["Gender"]=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,5000


In [35]:
#Combining one-hot encoded  and Label encoded columns with input Data
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,5000,1.0,0.0,0.0


##### Scaling Pickle

In [36]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.65923237,  1.00150113,
        -0.57946723, -0.57638802]])

##### Predictions

In [37]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 130ms/step


array([[0.03985695]], dtype=float32)

In [38]:
#prediction Probability
prediction_proba = prediction[0][0]
prediction_proba

0.03985695

In [39]:
if prediction_proba > 0.5:
    print('The customer is likely to Churn')
else:
    print("Customer not likely to Churn")

Customer not likely to Churn


## STREAMLIT CODE

In [40]:
'''
import streamlit as st
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

#Load the Trained Model

model = tf.keras.models.load_model('model.h5')

##load the encoder and Scler
with open('Label_Encoder_Gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

## Streamlit app

st.title("Customer Churn Prediction")

#User Input
geography = st.selectbox('Geography',onehot_encoder_geo.categories_[0])
gender=st.selectbox('Gender',label_encoder_gender.classes_)
age=st.slider('Age',18,92)
balance=st.number_input("Balance")
credit_score=st.number_input("Credit Score")
estimated_salary = st.number_input("Estimated Salary")
tenure = st.slider('Tenure',0,10)
num_of_products = st.slider('Number of Products',1,4)
has_cr_card=st.selectbox("Has Credit card",[0,1])
is_active_member = st.selectbox("Is Active Member",[0,1])

#Prepairing Input Data in Dict format
input_data=pd.DataFrame({
    'CreditScore':[credit_score],
    'Gender':[label_encoder_gender.transform([gender])[0]],
    'Age':[age],
    'Tenure':[tenure],
    'Balance':[balance],
    'NumOfProducts':[num_of_products],
    'HasCrCard':[has_cr_card],
    'IsActiveMember':[is_active_member],
    'EstimatedSalary':[estimated_salary]
})

#One Hot Encoding the Geography Column
geo_encoded = onehot_encoder_geo.transform([[geography]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

#Combine one_hot Encoded columns with input Data
input_data = pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)

#Scaling the Input Data
input_data_scaled = scaler.transform(input_data)

#Predict Churn
prediction = model.predict(input_data_scaled)
prediction_prob = prediction[0][0]

st.write(f'Churn Probability : {prediction_prob:.2f}')

if prediction_proba > 0.5:
    st.write('The customer is likely to Churn')
else:
    st.write("Customer not likely to Churn")

    
'''

'\nimport streamlit as st\nimport numpy as np\nimport tensorflow as tf\nimport pandas as pd\nimport pickle\nfrom sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder\n\n#Load the Trained Model\n\nmodel = tf.keras.models.load_model(\'model.h5\')\n\n##load the encoder and Scler\nwith open(\'Label_Encoder_Gender.pkl\',\'rb\') as file:\n    label_encoder_gender=pickle.load(file)\n\nwith open(\'one_hot_encoder_geo.pkl\',\'rb\') as file:\n    onehot_encoder_geo=pickle.load(file)\n\nwith open(\'scaler.pkl\',\'rb\') as file:\n    scaler=pickle.load(file)\n\n## Streamlit app\n\nst.title("Customer Churn Prediction")\n\n#User Input\ngeography = st.selectbox(\'Geography\',onehot_encoder_geo.categories_[0])\ngender=st.selectbox(\'Gender\',label_encoder_gender.classes_)\nage=st.slider(\'Age\',18,92)\nbalance=st.number_input("Balance")\ncredit_score=st.number_input("Credit Score")\nestimated_salary = st.number_input("Estimated Salary")\ntenure = st.slider(\'Tenure\',0,10)\nnum_of_